In [1]:
import matplotlib.pyplot as plt  
import pandas as pd  
import numpy as np 
import seaborn as sns
import os, sys
from hsbmpy import get_file, define_labels, get_cluster_given_l
from sklearn.metrics import homogeneity_completeness_v_measure
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
level = 2
#setup = 'oversigma_10tissue'
#label = 'disease_type'
label='primary_site'
#label = 'uniq'
L = 4
#labels = ['primary_site', 'disease_type']
#labels = ['primary_site', 'secondary_site']
directory=r"/home/fvalle/phd/datasets/tcga/TCGA/"
#directory=r"/home/fvalle/phd/results/altmann"
os.chdir(directory)

In [3]:
df = pd.read_csv("mainTable.csv", index_col=[0], header=[0])
totalobjcets = len(df.columns)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19369 entries, ENSG00000167578 to ENSG00000181518
Columns: 8969 entries, e1877a07-b5ec-4ada-89cb-525e5abf966a.htseq.counts.gz to 89e20bd9-15e5-48e6-9dfe-0b919c817216.htseq.counts.gz
dtypes: float64(8969)
memory usage: 1.3+ GB


In [4]:
df_files = pd.read_csv("files.dat", index_col=[0])
df_files.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, d1051cf3-e258-4250-b31e-26a1711c90ee.htseq.counts.gz to 984e48d4-d2d0-4dee-957a-20a9d1150781.htseq.counts.gz
Data columns (total 4 columns):
primary_site         100 non-null object
tcga_id              100 non-null object
disease_type         100 non-null object
primary_diagnosis    100 non-null object
dtypes: object(4)
memory usage: 3.9+ KB


In [5]:
df_files

,primary_site,tcga_id,disease_type,primary_diagnosis
file_name,,,,
d1051cf3-e258-4250-b31e-26a1711c90ee.htseq.counts.gz,Lung,TCGA-56-8308,Lung Squamous Cell Carcinoma,"Squamous cell carcinoma, NOS"
89e20bd9-15e5-48e6-9dfe-0b919c817216.htseq.counts.gz,Liver,TCGA-EP-A2KA,Liver Hepatocellular Carcinoma,"Hepatocellular carcinoma, NOS"
43428573-6606-4e04-bb8f-e4513dfe2556.htseq.counts.gz,Liver,TCGA-DD-AACP,Liver Hepatocellular Carcinoma,"Hepatocellular carcinoma, NOS"
101685f2-3131-40b2-9aa8-6458f0388b04.htseq.counts.gz,Liver,TCGA-DD-A11C,Liver Hepatocellular Carcinoma,"Hepatocellular carcinoma, NOS"
fa7877bb-9e46-4ecb-8c47-501893ce2410.htseq.counts.gz,Liver,TCGA-DD-A3A8,Liver Hepatocellular Carcinoma,"Hepatocellular carcinoma, NOS"
...,...,...,...,...
c2b14984-5f00-40a2-b977-990f30066906.htseq.counts.gz,Prostate,TCGA-EJ-7125,Prostate Adenocarcinoma,"Adenocarcinoma, NOS"
526b6171-5947-439d-b61f-4604309532e9.htseq.counts.gz,Prostate,TCGA-EJ-7797,Prostate Adenocarcinoma,"Adenocarcinoma, NOS"
cf4fa58c-803f-4195-bf42-13aa71403d16.htseq.counts.gz,Prostate,TCGA-ZG-A8QX,Prostate Adenocarcinoma,"Adenocarcinoma, NOS"


In [6]:
true_out = []
for sample in df.columns.values:
    try:
        true_out.append(get_file(sample, df_files)['primary_site'])
    except:
        print(sys.exc_info()[0])
        true_out.append('')

<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>


<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>


<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>


<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>


<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>
<class 'TypeError'>


In [7]:
scores = {}

In [8]:
try:
    with open("clustersizes.txt",'r') as f:
        xl = np.array(f.read().split(sep='\n'))[:-1].astype(int)
except:
        xl=np.linspace(2,15,14, dtype=int)

In [9]:
df.T.values.shape

(8969, 19369)

In [10]:
fig=plt.figure()
dend = shc.dendrogram(shc.linkage(df.T.values, method='ward'), leaf_rotation=90., leaf_font_size=8.,)
plt.xlabel("samples", fontsize=16)
plt.show()
fig.savefig("hierarchical_dendogram.pdf")

ValueError: The condensed distance matrix must contain only finite values.

<Figure size 432x288 with 0 Axes>

In [ ]:
#hierarchical
scores['hierarchical']={
    'h':[],
    'c':[],
    'V':[]
}
print("hierarchical")
os.system('mkdir -p hierarchical')
hierarchical_model = AgglomerativeClustering(n_clusters=1, affinity='euclidean', linkage='complete')  
for l,x in enumerate(xl):
    print("testing with %d clusters"%x)
    hierarchical_model.n_clusters=x
    out = hierarchical_model.fit_predict(df.T.values)
        
    #save clusters
    print("saving clusters")
    df_clusters = pd.DataFrame(index=np.arange(totalobjcets))
    for c in np.arange(out.max()+1)[::-1]:
        c_objects = df.columns[np.argwhere(out==c)].values.T[0]
        df_clusters.insert(0,"Cluster %d"%(c+1),np.concatenate((c_objects,[np.nan for _ in np.arange(totalobjcets-len(c_objects))])))
    df_clusters.dropna(axis=0,how='all', inplace=True)
    df_clusters.to_csv("hierarchical/hierarchical_level_%d_clusters.csv"%(l), index=False, header=True)
    
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['hierarchical']['h'].append(score[0])
    scores['hierarchical']['c'].append(score[1])
    scores['hierarchical']['V'].append(score[2])
    
pd.DataFrame(data=scores['hierarchical']).to_csv("hierarchical.scores", header=True, index=False)

## LDA

In [ ]:
with open("topicsizes.txt",'r') as f:
    tl = np.array(f.read().split(sep='\n'))[:-1].astype(int)

In [ ]:
#xl = [2,3,4,5]
tl = xl
Sigmas = []

In [ ]:
scores['lda']={
    'h':[],
    'c':[],
    'V':[]
}
print("lda")
os.system('mkdir -p lda')
lda = LatentDirichletAllocation(n_components=1, 
                                    random_state=42, 
                                    n_jobs=12, 
                                    learning_method='online', 
                                    max_doc_update_iter=5,
                                    max_iter=5,
                                    topic_word_prior=1,
                                    doc_topic_prior=1,
                                    verbose=2)
for l,x in enumerate(xl):
    #lda
    ntopic = tl[l]
    #ntopic = x
    print("testing with %d clusters and %d topics"%(x,ntopic))
    lda.n_components=ntopic
    print(lda)
    topics = lda.fit_transform(df.T.values)
    
    #save word distr
    print("saving word-distr")
    df_word_distr = pd.DataFrame(data=lda.components_.T, index=df.index, columns=["Topic %d"%(t+1) for t in np.arange(ntopic)])
    df_word_distr.to_csv("lda/lda_level_%d_word-dist.csv"%l, index=True, header=True)
    
    #save topic distr
    print("saving topic-distr")
    df_topic_distr = pd.DataFrame(data=topics, columns=["Topic %d"%(t+1) for t in np.arange(ntopic)])
    df_topic_distr.insert(0,'i_doc',np.arange(len(df.columns)))
    df_topic_distr.insert(1,'doc',df.columns)
    df_topic_distr.to_csv("lda/lda_level_%d_topic-dist.csv"%l, index=False, header=True)
    
    #save clusters
    print("saving clusters")
    df_clusters = pd.DataFrame(index=np.arange(totalobjcets))
    #cluster = AgglomerativeClustering(n_clusters=x, affinity='euclidean', linkage='ward')  
    #out = cluster.fit_predict(topics)
    out = np.argmax(topics, axis=1)
    
    for c in np.arange(out.max()+1)[::-1]:
        c_objects = df.columns[np.argwhere(out==c)].values.T[0]
        df_clusters.insert(0,"Cluster %d"%(c+1),np.concatenate((c_objects,[np.nan for _ in np.arange(totalobjcets-len(c_objects))])))
    df_clusters.dropna(axis=0,how='all', inplace=True)
    df_clusters.to_csv("lda/lda_level_%d_clusters.csv"%(l), index=False, header=True)
    
    #metrics
    print("saving metrics")
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['lda']['h'].append(score[0])
    scores['lda']['c'].append(score[1])
    scores['lda']['V'].append(score[2])
    
    #save dl
    Sigmas.append(-lda.score(df.values.T))
    
pd.DataFrame(data=scores['lda']).to_csv("%s/lda.scores"%directory, header=True, index=False)

In [ ]:
Sigmas

In [ ]:
homogeneity_completeness_v_measure(true_out, out)

## hierachical on Altmann's output

In [ ]:
hiermodel = AgglomerativeClustering(n_clusters=10, affinity='euclidean', linkage='ward')

In [ ]:
with open('clustersizes.txt') as f:
    xl=np.array(f.read().split('\n')[:-1]).astype(int)

In [ ]:
os.system("mkdir -p hierhsbm")

In [ ]:
true_out = []
for sample in pd.read_csv("%s/%s_level_%d_topic-dist.csv"%('topsbm','topsbm',0), index_col=1).drop('i_doc', axis=1).index.values:
    try:
        true_out.append(get_file(sample, df_files)['primary_site'])
    except:
        print(sys.exc_info()[0])
        true_out.append('')

In [ ]:
scores['hierhsbm']={
    'h':[],
    'c':[],
    'V':[]
}
for l,n_clusters in enumerate(xl):
    print("Fitting level %d with %d clusters"%(l, n_clusters))
    df_topics = pd.read_csv("%s/%s_level_%d_topic-dist.csv"%('topsbm','topsbm',l), index_col=1).drop('i_doc', axis=1)
    df_clusters = pd.DataFrame(columns=["Cluster %d"%c for c in np.arange(n_clusters)+1])
    hiermodel.n_clusters=n_clusters
    out = hiermodel.fit_predict(df_topics.values)  
    for c in np.arange(out.max()+1)[::-1]:
        c_objects = df_topics.index[np.argwhere(out==c)].values.T[0]
        df_clusters["Cluster %d"%(c+1)]=np.concatenate((c_objects,[np.nan for _ in np.arange(len(df_topics.index)-len(c_objects))]))
    df_clusters.dropna(axis=0,how='all', inplace=True)
    df_clusters.to_csv("hierhsbm/hierhsbm_level_%d_clusters.csv"%(l), index=False, header=True)
    #metrics
    print("saving metrics")
    score = (homogeneity_completeness_v_measure(true_out, out))
    scores['hierhsbm']['h'].append(score[0])
    scores['hierhsbm']['c'].append(score[1])
    scores['hierhsbm']['V'].append(score[2])
    
pd.DataFrame(data=scores['hierhsbm']).to_csv("%s/hierhsbm.scores"%directory, header=True, index=False)